# Import

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score

In [2]:
from Saves.HelperFunctions import *
from Preprocessing import preprocess

In [3]:
train_data = pd.read_csv('Data/train.csv')
test_data = pd.read_csv('Data/test.csv')
# test_data = None

In [4]:
train_data.head(2)

,SampleID,Discussion,Category
0,1,"Without sitting down and doing it manually, yo...",Sports
1,2,All your Search ends with this link.,STEM


In [5]:
# test_data.head(3)

# Preprocessing

In [6]:
pre_method = 2
X_train_padded, Y_train, X_test_padded, num_unique_words, max_sequence_length = preprocess(train_data=train_data,
                                                                                            test_data=test_data, 
                                                                                            pre_method=pre_method,
                                                                                            fx_opt=2)

Drop Nan...
	train_data.shape before (24989, 3)
	train_data.shape after (24646, 3)
start preprocessing...
Encoding Y_train...
	Num of Unique words: 20000
Tokenizer...
finish fitting...
tokenizer saved
avg_sequence_length = 34
max_sequence_length = 759


In [7]:
print(f"X_train.shape: {X_train_padded.shape}")
print(f"X_test.shape: {X_test_padded.shape}")
print(f"Y_train.shape: {Y_train.shape}")
# X_train.shape: (24646, 100)
# X_test.shape: (10557, 100)
# Y_train.shape: (24646,)

X_train.shape: (24646, 100)
X_test.shape: (10557, 100)
Y_train.shape: (24646,)


In [8]:
Y_train.head(3)

0    1
1    4
2    4
Name: Category, dtype: int64

# RNN

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout
# from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU

In [9]:
print(f"num_unique_words: {num_unique_words}")
print(f"max_sequence_length: {max_sequence_length}")

num_unique_words: 20000
max_sequence_length: 100


In [10]:
num_classes = 5
# Define the model
model = Sequential()

# Embedding layer to convert word indices into dense vectors of fixed size
model.add(Embedding(input_dim=num_unique_words, output_dim=128, input_length=max_sequence_length))

if False:
    # RNN layer (SimpleRNN)
    model.add(SimpleRNN(units=64, return_sequences=False))
elif True:
    # GRU layer
    model.add(GRU(units=64, return_sequences=False, activation='tanh'))

# Dropout for regularization
model.add(Dropout(0.5))

# Fully connected layer for classification
model.add(Dense(5, activation='softmax'))  # Adjust output units for the number of classes

# Explicitly build the model
model.build(input_shape=(None, max_sequence_length))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

d:\Programing\Python\Anaconda\envs\deeplearning\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,597,573 (9.91 MB)

 Trainable params: 2,597,573 (9.91 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Assuming Y_train contains labels in integer form (e.g., [0, 1, 2, 3, 4])
Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)

In [12]:
epochs = 3
batch_size = 32
validation_split = 0.2

In [14]:
model.fit(X_train_padded, Y_train_categorical, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

Epoch 1/3
617/617 ━━━━━━━━━━━━━━━━━━━━ 74s 120ms/step - accuracy: 0.6850 - loss: 0.8337 - val_accuracy: 0.6692 - val_loss: 0.8700
Epoch 2/3
617/617 ━━━━━━━━━━━━━━━━━━━━ 76s 123ms/step - accuracy: 0.8141 - loss: 0.5509 - val_accuracy: 0.6649 - val_loss: 0.9316
Epoch 3/3
617/617 ━━━━━━━━━━━━━━━━━━━━ 54s 87ms/step - accuracy: 0.8668 - loss: 0.4071 - val_accuracy: 0.6564 - val_loss: 1.0315


## Evaluation

### Train

In [ ]:
# from tensorflow.keras.models import load_model
# model_loaded = load_model('Models\\RNN\\GRU-e10-a81.h5')

In [15]:
# Predict the labels for the test set
train_predictions = model.predict(X_train_padded)

# If it's a multi-class classification task, get the predicted class for each sample
Y_train_pred = np.argmax(train_predictions, axis=1)

771/771 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step


In [16]:
train_accuracy = accuracy_score(Y_train_pred, Y_train)
print(f"Train Accuracy: {train_accuracy}")

Train Accuracy: 0.8573399334577619


In [17]:
file_name = f'GRU-m2-e9-a{int(train_accuracy*100)}'
model.save(f'Models/RNN/{file_name}.h5')
file_name

'GRU-m2-e9-a85'

### Test

In [18]:
# Predict the labels for the test set
test_predictions = model.predict(X_test_padded)

# If it's a multi-class classification task, get the predicted class for each sample
Y_test_pred = np.argmax(test_predictions, axis=1)

330/330 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step


In [19]:
file_name = f'GRU-e9'
save_csv(data=Y_test_pred, file_name=file_name, header=['SampleID', 'Category'], numbering=True)

Data saved to Saves/GRU-e9.csv
